# Importing

In [1]:
import tensorflow as tf
import numpy as np
import sys
import pandas as pd
import cv2
import time
from tqdm import tqdm
import string

import plotly.offline as py
import plotly.graph_objs as go

c:\miniconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Data Processing

In [4]:
Trainfolder = "train/train/"
Testfolder = "test/"
Labelsfolder = "labels.csv/labels.csv"
Samplefolder = "sample_submission.csv/sample_submission.csv"

In [ ]:
Training = pd.read_csv(Labelsfolder)

In [ ]:
Training['image_path'] = Training.apply(lambda x: (Trainfolder + x["id"] + ".jpg" ), axis=1)

In [5]:
test = pd.read_csv(Samplefolder)
test['image_path'] = test.apply(lambda x: Testfolder + x["id"] + '.jpg', axis=1)

In [7]:
def get_pixels(dim, path ):
    image = cv2.imread(path, 1)
    resized = cv2.resize(image, (dim, dim))
    return resized

def show_image(img):
    cv2.imshow('gos', img)
    cv2.waitKey()

def normalize(img):
    img  = (img-img.min())/(img.max()-img.min())
    return np.array(img, dtype= 'float32')

def save_in_hdf(df, batches = 1, s = 'X'):
    
    if s == 'test': np.save('testdata', df )
    
    else:
        if batches == 1 :
            if s == 'X': np.save('traindata{}'.format(s), df )
            else:

                with pd.HDFStore("data{}.h5".format(s)) as hdf:
                    hdf.put("training{}".format(s), df)

        else:

            df1 = np.array_split(df, batches)


            for i in range(0, batches):
                if s == 'X': np.save('traindata{}'.format(i+1), df1[i] )

                else:
                    with pd.HDFStore("data{}{}.h5".format(i+1, s)) as hdf:
                        hdf.put("training{}{}".format(i+1,s), df1[i])

                print("Saved batch number {}".format(i+1))

                
def load_hdf( batch_id, s = 'X'):
    if s == 'test': df3 = np.load('testdata')
    else:
        if s == 'X': df3 = np.load('traindata{}.npy'.format(batch_id))       
        else: df3 = pd.read_hdf('data{}{}.h5'.format(batch_id,s),'training{}{}'.format(batch_id,s))

        print("Loaded batch number {}".format(batch_id))
    
    return df3

#### Example

In [ ]:
img = get_pixels(320, 'train/train/000bec180eb18c7604dcecc8fe0dba07.jpg' )

In [ ]:
image = cv2.imread('train/train/000bec180eb18c7604dcecc8fe0dba07.jpg', 1)


In [ ]:
sys.getsizeof(img)

In [ ]:
img2 = normalize(img)

## Reading, Preprocessing and Saving

In [ ]:
DIM = 99
channels = 3
lst = []
for path in tqdm(Training['image_path']):
    lst.append(normalize(get_pixels(DIM,path)))
    
Training["Data"] = lst
Training = pd.get_dummies(Training, columns=['breed'])


In [9]:
DIM = 99
channels = 3
lst2 = []
for path in tqdm(test['image_path']):
    lst2.append(normalize(get_pixels(DIM,path)))

    
test["Data"] = lst2
x_test = np.array(lst2)

In [ ]:
Xtrain = np.array(lst)
del lst
dropped = []
for i in range(0,3):
    dropped.append(Training.columns[i])
    
Ytrain = Training.drop(dropped, axis=1, inplace=False)

In [ ]:
dic = {}
num_classes = len(Ytrain.columns)
for i in range(0, num_classes-1):
    l = list(Ytrain.columns[i])
    l[0:6] = []
    elem2 = "".join(l)
    dic[Ytrain.columns[i]] = elem2
    
Ytrain = Ytrain.rename(dic, axis = "columns")

In [ ]:
nb_batches = 1
save_in_hdf(Xtrain, nb_batches, 'X')
save_in_hdf(Ytrain, nb_batches, 'Y')


In [12]:
save_in_hdf(x_test, 1, 'test')

In [22]:
df = pd.read_csv('prova.csv', header = None, names = ['col1', 'col2'])